<a href="https://colab.research.google.com/github/Timliuw/DP-Tim/blob/main/CI_Tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import pandas as pd
import uuid
import psutil
from functools import lru_cache

In [17]:
# Google Spreadsheet
import gspread
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files
# uploaded = files.upload()
import os
os.listdir()

scope = [
    "https://spreadsheets.google.com/feeds",  # Google Sheets API
    "https://www.googleapis.com/auth/drive"   # Google Drive API
]

# Trigger the file upload dialog
try:
    creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
except:
   uploaded = files.upload()

creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open_by_key("1yVVt5ACdBbjsZ_7IiRkvPGIHmpKe8mFXcvyL0PVP2cE")

worksheet1 = spreadsheet.get_worksheet(0)
worksheet = worksheet1
worksheet2 = spreadsheet.get_worksheet(1)

def get_color(curr_value, prev_value):
    if curr_value < 0.6 * prev_value:
        return {"red": 0.6, "green": 1.0, "blue": 0.6}  # very green
    elif curr_value < prev_value:
        return {"red": 0.8, "green": 1.0, "blue": 0.8}  # green
    elif curr_value > 1.4 * prev_value:
        return {"red": 1.0, "green": 0.6, "blue": 0.6}  # very red
    elif curr_value > prev_value:
        return {"red": 1.0, "green": 0.8, "blue": 0.8}  # red
    return None

def write_to_sheet(worksheet, row_data, output_mode=0, compare=False):
    # Append the new row to the sheet
    try:
        worksheet.append_row(row_data)
    except Exception as e:
        print(f"Error occurred: {e}")
        print(f"Row data: {row_data}")
    # Format colour according to the comparing results
    if compare:
        pre_row_index = -2
        if (output_mode == 2): pre_row_index = -18
        sheet_data = worksheet.get_all_values()
        last_row_index = len(sheet_data)
        if last_row_index > 1:
            last_row = sheet_data[-1]
            prev_row = sheet_data[pre_row_index]
            for col_index, (curr_value, prev_value) in enumerate(zip(last_row, prev_row), start=1):
                try:
                    curr_value = float(curr_value)
                    prev_value = float(prev_value)
                    if (worksheet.row_values(1)[col_index-1] == "est median"):
                      prev_value = abs(true_med - prev_value)
                      curr_value = abs(true_med - curr_value)
                    color = get_color(curr_value, prev_value)
                    if color:
                        worksheet.format(f"{chr(64 + col_index)}{last_row_index}", {"backgroundColor": color})
                except:
                    continue

In [18]:
def discretize(D, b):
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD

def discrete(D):
    n = len(D)
    newD = np.zeros(n)
    previous = -1
    for i in range(n):
        if D[i]!=previous:
            newD[i] = n*D[i]
            #print(newD[i])
            previous = D[i]
        elif D[i]==previous:
            newD[i] = newD[i-1]+1

    return newD

def count(D, a):
    counter = 0
    n = len(D)
    low = 0
    up=n-1
    mid = int((low+up)/2)
    while True:
        if D[mid]>a:
            up=mid
            mid = int((low+up)/2)
        if D[mid]<a:
            low=mid
            mid = int((low+up)/2)
        if D[mid]==a:
            i=0
            while D[mid+i]==a:
                i+=1
            return mid+i

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else:
                low = mid
            mid = (high+low)/2

        return high
    if y==1/2:
        return 0

def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

In [19]:
# Their algorithm
def constructu(eps, a,b, D,sensitivity = 0.5):
    # global u
    # global l
    # global weight
    #[a,b] denotes range
    n = len(D)

    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#corresponding data
    l[0] = a
    for i in range(n+1): #i=rank-1
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]

    l[n+1] = b
    u[n+1] = -n-1-u[0]
    weight = []
    # l[i] corresponds to data in D with rank i
    for i in range(int(n/2)+1): # build weight[0...n/2]
        weight.append((l[i+1]-l[i])*math.pow(np.e, eps*u[i]/(2*sensitivity))) # weight[i]=(l[i+1]-l[i])...
    weight.append(1) # weight[n/2+1] = 1
    for i in range(int(n/2)+1,n+1): # build weight[n/2+2..n+1]
        weight.append((l[i]-l[i-1])*math.pow(np.e, eps*u[i]/(2*sensitivity))) # weight[i+1]=(l[i]-l[i-1])...
        # which means weight[i] corresponds to interval [ l[i-1],l[i-2] ) for i > int(n/2)+1
        # should be:
        # weight.append((l[i+1]-l[i])*math.pow(np.e, eps*u[i]/(2*sensitivity))) # weight[i+1]=(l[i+1]-l[i])...
    totalWeight = sum(weight)
    weight = weight/totalWeight
    # if show_EM_median_u:
    #   print("totalWeight",totalWeight)
    #   df["Their u"] = list(u)
    #   df['Their weights'] = weight
    return u, l, weight

def EMMedian_new(l,weight):
    i = np.random.choice(list(range(len(l))), p=weight) # choose index i from 0 to n+1
    # pick i with weight[i]
    # only return int
    if i==int(n/2)+1:
        return int(l[i]/n)
    if i<int(n/2)+1:
        return int(np.random.randint(l[i], l[i+1],dtype=np.int64) / n)
    if i>int(n/2)+1:
        return int(np.random.randint(l[i-1], l[i],dtype=np.int64) / n) # bug?

def constructu_CI(eps, beta, N, u, l,sensitivity = 0.5):
    # global u1
    # global weight1
    # global u2
    # global weight2
    # global factor
    if show_EM_median_u:
      return [],[],[],[],0
    factor = int(8/eps*np.log(4*n*N/beta))
    print("factor is "+str(factor))
    #u1: left util
    #u2: right util
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2)+1:
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)

    idx = int(n/2)+1-factor
    weight1 = []
    for i in range(idx):
        weight1.append((l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/(4*sensitivity)))
    weight1.append(1)
    for i in range(idx,n+1):
        weight1.append((l[i]-l[i-1])*math.pow(np.e, eps*u1[i]/(4*sensitivity)))
    totalWeight1 = sum(weight1)
    weight1 = weight1/totalWeight1

    idx = int(n/2)+1+factor
    idx = min(int(n/2)+1+factor,n+1)
    weight2 = []
    for i in range(idx):
        weight2.append((l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/(4*sensitivity)))
    weight2.append(1)
    for i in range(idx,n+1):
        weight2.append((l[i]-l[i-1])*math.pow(np.e, eps*u2[i]/(4*sensitivity)))

    totalWeight2 = sum(weight2)
    weight2 = weight2/totalWeight2
    return u1,weight1,u2,weight2,factor

def EMMedianCI(l,weight,weight1, weight2,factor):
    est = EMMedian_new(l,weight)
    if show_EM_median_u:
      return 0,0,0,est
    i1 = np.random.choice(list(range(len(l))), p=weight1)
    if i1==int(n/2)+1-factor:
        x1= int(l[i1]/n)
    if i1<int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1], l[i1+1],dtype=np.int64)/n)
    if i1>int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1-1], l[i1],dtype=np.int64)/n)

    i2 = np.random.choice(list(range(len(l))), p=weight2)
    if i2==int(n/2)+1+factor:
        x2= int(l[i2]/n)
    if i2<int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2], l[i2+1],dtype=np.int64)/n)
    if i2>int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2-1], l[i2],dtype=np.int64)/n)


    indicator=0
    if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
        indicator=1

    return x1,x2 ,indicator,est

In [20]:
eps = []
beta = []
D = []
newD = []
domain_size = 4000
num_repeat = 100
b_list_step = 1


In [21]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key):
        if key in self.cache:
            self.cache.move_to_end(key)
            return self.cache[key]
        return None

    def set(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

rank_dict_record = LRUCache(capacity=100000)
rank_dict_record_discretized = LRUCache(capacity=100000)

In [22]:
from json.encoder import INFINITY
# Our algorithm
#1) Learn noisy median o of distribution using Exponential Mechanism
def EM_median(eps,beta,D,domain_size,use_discretized = False):
    interval_prob = []
    rank_dict = {}
    # D = np.array([lowest]) + D
    vals, counts = np.unique(D,return_counts=True)
    rank = 0
    n = len(D)
    last_val = vals[-1]
    if use_discretized:
      last_val += n
    else:
      last_val += 1
    vals = vals.tolist() + [int(last_val)]
    counts = counts.tolist() + [0]
    cdf = 0.0
    weights = []
    u = []

    for i in range(0,len(vals)-1):
      rank += counts[i]
      rank_dict[vals[i]] = rank
      utility = -1.0* np.abs(rank - n/2)
      u.append(utility) # u[i] = utility, corresponding to vals[i]
      p = (vals[i+1] - vals[i])* math.pow(np.e, eps*utility/2)
      cdf += p
      weights.append(p) # weight[i] = p, corresponding to interval [vals[i],vals[i+1])
      if use_discretized == 0:
        for j in range(counts[i]-1):
          weights.append(".")
          u.append(".")
      interval_prob.append(cdf) # interval[i-1] = cdf,corresponding to interval (val[i-1],val[i]]
    rank_dict[vals[-1]] = rank
    # print("our weight len",len(weight))
    rand_val = cdf * np.random.uniform(0,1)
    rank_o = 0
    # print("total weights:",sum(weights))
    if (use_discretized == 0):
      weight_name = 'our weights'
      u_name = 'our u'
    else:
      weight_name = 'our weights(discre)'
      u_name = 'our u(discre)'

    for i in range(0,len(vals)-1):
        if rand_val <= interval_prob[i]:
          o = int(np.random.randint(vals[i], vals[i+1],dtype=np.int64)) # [vals[i],vals[i+1])
          rank_o = rank_dict[vals[i]]
          break
    if show_EM_median_u:
      totalWeight = 0
      for weight in weights:
        try:
          totalWeight +=weight
        except:
          continue
      # print("totalWeight",totalWeight)
      std_weight = []
      for weight in weights:
          try:
              std_weight.append(weight / totalWeight)
          except Exception as e:
              std_weight.append(".")
      df[u_name] = ["."] + u + ["."]
      df[weight_name] = ["."] + weights + ["."]
      df["std_"+weight_name] = ["."] + std_weight + ["."]
    return o,rank_o,rank_dict

#2)
def find_rank(rank_dict, val, vals=None, ranks=None):
    if use_discretized == 0:
        cached = rank_dict_record.get(val)
        if cached is not None:
            return cached
    else:
        cached = rank_dict_record_discretized.get(val)
        if cached is not None:
            return cached
    if vals is None:
        vals = list(rank_dict.keys())
        ranks = list(rank_dict.values())
    if val <= vals[0]:
        result = 0
    elif val > vals[-1]:
        result = ranks[-1]
    else:
        left, right = 0, len(vals) - 1
        while left <= right:
            mid = (left + right) // 2
            if val > vals[mid]:
                left = mid + 1
            else:
                right = mid - 1
        result = ranks[left - 1] if left > 0 else 0

    if use_discretized == 0:
        rank_dict_record.set(val, result)
    else:
        rank_dict_record_discretized.set(val, result)
    return result



def SVT_median(o,rank_o,rank_dict,eps,D,domain_size,T,b_list):
    # print("T: ", T)
    noisy_T = T + np.random.laplace(2/eps)
    sensitivity = 1
    bounds = domain_size
    #print(min(o+bounds,domain_size-1))
    for b in b_list: #range(0,b_max,step):
        perturbed_value = min(abs(find_rank(rank_dict, o+b)-rank_o),abs(find_rank(rank_dict, o-b)-rank_o)) + np.random.laplace(2*sensitivity/eps)
        # print("o:",o,"rank_o",rank_o)
        # print("find_rank(rank_dict, o)",find_rank(rank_dict, o))
        # print(min(abs(find_rank(rank_dict, o+b)-rank_o),abs(find_rank(rank_dict, o-b)-rank_o)))
        if perturbed_value > noisy_T:
            bounds = b
            # print("b:",b,"perturbed_value:",perturbed_value,"noisy_T:",noisy_T)
            break
    #print(max(o-bounds,0))

    # print(perturbed_value)
    # result = max(o-bounds,0),min(o+bounds,domain_size-1) # bug? consider o=max(D)
    # result = max(o-bounds,0),min(o+bounds,D[-1]-1) # fixed?
    result = max(o-bounds,0),min(o+bounds,D[-1])
    # result = max(o-bounds,D[0]),min(o+bounds,D[-1]) #

    # print(b,result)
    return result

def EMCI(D,o,rank_o,rank_dict,eps,b_list,C1,C2,domain_size,sl_in_u=False,precomputed_vals=None,precomputed_ranks=None):
  u = []
  fb_list = []
  weights = []
  interval_prob = []
  cdf = 0.0
  l = 1
  n = len(D)
  if len(b_list) >= 1:
    step_size = b_list[0]
  # b_list = list(b_list)

  for i in range(0,len(b_list)):
    b = b_list[i]
    # if (o+b not in rank_dict_record.keys()):
    #   print("o+b not in rank_dict_record.keys()",b)
    # if (o-b not in rank_dict_record.keys()):
    #   print("o-b not in rank_dict_record.keys()",b)
    right_bound = abs(find_rank(rank_dict,o+b,vals=precomputed_vals,ranks=precomputed_ranks)-rank_o)
    left_bound = abs(find_rank(rank_dict,o-b,vals=precomputed_vals,ranks=precomputed_ranks)-rank_o)
    fb = min(left_bound,right_bound)
    # print("b:",b/n,"fb:",rank_o,left_bound, right_bound)
    fb_list.append(fb)

    if sl_in_u: utility = -1.0 * abs(fb - C1 - C2)
    else: utility = -1.0 * abs(fb - C1 - C2 - step_size*l)
    u.append(utility)

  sensitivity = 1
  for i in range(0, len(b_list)):
    utility = u[i]
    # eps_utility = np.clip(eps * utility / 2, -700, 700)
    eps_utility = eps * utility / sensitivity # u is monotonic
    # p = (b_list[i] - b_list[i-1]) * np.exp(eps_utility)
    # p = math.pow(np.e, eps_utility)
    p = np.exp(eps_utility)
    cdf += p
    weights.append(p)
    interval_prob.append(float(cdf))
  # print("cdf",cdf)
  if show_EMCI_u:
    original_b = [b/n for b in b_list]
    df['b'] = original_b
    df['b'] = b_list
    df['fb'] = ['.']+fb_list
    df['utility'] = ['.']+u
    df['weight'] = ['.']+weights
    df['interval_prob'] = ['.']+interval_prob

  rand_val = cdf * np.random.uniform(0,1)
  # print("rand_val:",rand_val)
  bounds = domain_size
  for i in range(0,len(b_list)-1):
    b = b_list[i+1]
    if rand_val <= interval_prob[i]:
      # bounds = int(np.random.randint(b_list[i]+1, b_list[i+1]+1,dtype=np.int64))
      bounds = b
      break
  # print("bounds:",bounds)
  # print(o,o+bounds)
  if sl_in_u:
    result = max(o-bounds,0),min(o+bounds,D[-1])
  else:
    result = max(o-bounds-step_size*l,0),min(o+bounds+step_size*l,D[-1])
  # result = max(o-bounds-step_size,0),min(o+bounds,D[-1])
  return result


def Their_EMMedian_modified(l,weight,D):
    n = len(D)
    vals, counts = np.unique(D,return_counts=True)
    rank_dict = {}
    rank_dict_record = {}
    rank = 0
    last_val = vals[-1]
    if use_discretized:
      last_val += n
    else:
      last_val += 1
    vals = vals.tolist() + [int(last_val)]
    counts = counts.tolist() + [0]
    for i in range(0,len(vals)-1):
        rank += counts[i]
        rank_dict[vals[i]] = rank
    rank_dict[vals[-1]] = rank
    i = np.random.choice(list(range(len(l))), p=weight)
    # only return int
    val = 0
    if i==int(n/2)+1:
      val = int(l[i])
    if i<int(n/2)+1:
      val = int(np.random.randint(l[i], l[i+1],dtype=np.int64))
    if i>int(n/2)+1:
      val = int(np.random.randint(l[i-1], l[i],dtype=np.int64))
      i=i-1
    # print("est_med:",val/n)
    return val,i,rank_dict #rank_dict is from global variable

##### main function ######

def our_CI(D,domain_size,b_list,med_method=0,CI_method=0,l=[],weight=[],sl_in_u=False):
    if len(b_list) >= 1:
      step_size = b_list[0]
    m = len(b_list)
    alpha = (8*np.log(m)+np.log(2/beta_2)) / eps_2
    C = (2/eps_1) *(np.log(domain_size/beta_1)) # γ
    C2 = (1/eps_2) *(np.log(domain_size/(b_list_step*beta_2)))
    # print("C1=",C,"C2=",C2)
    #print("alpha", alpha)
    T = C + alpha
    if (med_method==0):
      o,rank_o,rank_dict = EM_median(eps_1,beta_1,D,domain_size) #our EM
    elif (med_method==1):
      o,rank_o,rank_dict = Their_EMMedian_modified(l,weight,D) #their EM
    if (CI_method==0):
      lower,upper = SVT_median(o,rank_o,rank_dict,eps_2,D,domain_size,T,b_list)
    elif (CI_method==1):
      precomputed_vals = list(rank_dict.keys())
      precomputed_ranks = list(rank_dict.values())
      lower,upper = EMCI(D,o,rank_o,rank_dict,eps_2,b_list,C,C2,domain_size,sl_in_u,precomputed_vals,precomputed_ranks)
    else:
      lower,upper = 0,0 # do not run CI algorithm
    #print("median:",o)
    #print("The bounds are:")
    #print('[{},{}]'.format(lower,upper))
    indi = -1 #if CI contains true med
    if (use_discretized == 1):
      if lower < D[0]: lower = D[0]
      if upper > D[-1]: upper = D[-1]
      lower,upper = int(lower/n),int(upper/n)
      o = int(o/n)

    # if true_med < upper and true_med > lower:
    if true_med <= upper and true_med >= lower: #new
        indi = 1
    else:
        indi = 0
    return lower,upper,indi,o


In [23]:
def test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D,compare=False,output=[],med_method=0,CI_method=0,name="",sl_in_u = False):
  # med_method: 0 use our EM
  #      : 1 use their EM (use_discretized must = 1)
  domain_size = int(max(D)-min(D))
  if (use_discretized == 1):
    # b_list_step *= n
    b_list = range(b_list_step, b_list_step*int(domain_size/b_list_step)+1,b_list_step)

    u,l,weight = [],[],[]
    if (len(b_list) == 0): b_list = range(0,2)
    if (med_method == 1):
      rad = 10000000
      left = -rad
      u,l,weight=constructu(eps_1,left*n,rad*n, D,sensitivity = 1)
  else:
    # b_list = range(0, domain_size, b_list_step)
    b_list = range(b_list_step, b_list_step*int(domain_size/b_list_step)+1,b_list_step)
    l = 0
    weight = []
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  ranks = []
  lengths = []
  errors = []
  rank_errors_1 = [] # based on n/2
  rank_errors_2 = [] # based on find_rank(rank_dict,true_med)
  rank_CI_lengths = []
  # est_list = []
  med_rank = int(find_rank(rank_dict, true_med))
  # print("rank_dict:",rank_dict)
  # print("med_rank:",med_rank)
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      correct_count = 0

      for j in range(num_repeat):
          start = time.time()
          # xl,xr,indi,est = 0,0,0,0
          xl,xr,indi,est = our_CI(D,domain_size,b_list,med_method,CI_method,l,weight,sl_in_u)
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          lengths.append((xr-xl)/(2))
          # print(xr,xl)
          # interesting and subtle difference!
          # mid_value = (xr+xl)/2
          errors.append(abs(est-true_med))
          est_median.append(est)
          rank = int(find_rank(rank_dict, est))
          ranks.append(rank)
          rank_errors_1.append(int(abs(n/2 - rank)))
          rank_errors_2.append(int(abs(med_rank - rank)))
      original_errors = errors
      errors = np.array(errors, dtype=object)
      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]
      avg_error = np.average(errors)
      avgTime = sum(times)/len(times)

      # rank CI length
      rank_lower = int(find_rank(rank_dict,xl))
      rank_upper = int(find_rank(rank_dict,xr))
      rank_CI_length = (rank_upper-rank_lower)/2
      rank_CI_lengths.append(rank_CI_length)


      # output medians and error

      #download errors
      # df = pd.DataFrame(errors)
      # file_name = f"data_error.csv"
      # df.to_csv(file_name, index=False, header=False)
      # files.download(file_name)

      # median_error
      if np.all(errors == errors[0]):
          stddev_based_avg_err = errors[0]
          IQR_based_avg_err = errors[0]
      else:
          std_dev = np.std(errors)
          stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])
          q1 = np.percentile(errors, 25)
          q3 = np.percentile(errors, 75)
          iqr = q3 - q1
          lower_bound = q1 - 1.5 * iqr
          upper_bound = q3 + 1.5 * iqr
          IQR_based_avg_err = np.average(errors[(errors >= lower_bound) & (errors <= upper_bound)])
      # rank_error
      avg_rank_error_1 = np.average(rank_errors_1)
      avg_rank_error_2 = np.average(rank_errors_2)

      # avg rank CI length
      avg_rank_CI_length = np.average(rank_CI_lengths)

      if name=="":
        name = "our_CI"
        if (use_discretized == 1):
          name = "our_CI(discretized)"
          domain_size = int(domain_size/n)
          if (med_method == 1): name = "our_CI(their EM)"
      if (errorQuantile == 0): relative_CI_width = -1
      else: relative_CI_width = avgLength / errorQuantile
      print("last experiment CI:",xl,xr)
      print("avg estimated median value: "+ str(np.average(est_median)))
      print("avg median error: "+ str(avg_error))
      print("correct rate = ", correct_rate)
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(relative_CI_width))
      print("Average Time to find CI = " + str(avgTime))


      row_data = [
          name,
          data_uuid,
          n,
          domain_size,
          sens,
          beta[i],
          eps[i],
          num_repeat,
          true_med,
          np.average(est_median),
          correct_rate,
          avgLength,
          errorQuantile,
          avg_error,
          stddev_based_avg_err,
          IQR_based_avg_err,
          avg_rank_error_1,
          avg_rank_error_2,
          relative_CI_width,
          avg_rank_CI_length,
          avgTime,
          b_list_step,
          beta_1,
          beta_2,
          eps_1,
          eps_2
      ]
      if (output_mode == 0 or output_mode == 2):
        write_to_sheet(worksheet1, row_data, output_mode=output_mode, compare=compare)

      # New! Check all medians and errors
      if (output_mode == 1 or output_mode == 2):
        sorted_med = sorted(est_median)
        est_median.insert(0, "Our est_median")
        sorted_med.insert(0, "Sorted Our est_median")

        sorted_errors = sorted(original_errors)
        original_errors.insert(0, "Our errors")
        sorted_errors.insert(0, "Sorted Our errors")

        sorted_ranks = sorted(ranks)
        ranks.insert(0, "Our ranks")
        sorted_ranks.insert(0, "Sorted Our ranks")

        sorted_lengths = sorted(lengths)
        lengths.insert(0,"Our CI lengths")
        sorted_lengths.insert (0, "Sorted Our CI lengths")

        sorted_re_1 = sorted(rank_errors_1)
        sorted_re_2 = sorted(rank_errors_2)
        rank_errors_1.insert(0, "Our rank_errors(n/2)")
        rank_errors_2.insert(0, "Our rank_errors(find)")
        sorted_re_1.insert(0, "Sorted Our rank_errors(n/2)")
        sorted_re_2.insert(0, "Sorted Our rank_errors(find)")


        data_uid_row = ["data:", data_uuid, "method:", "Our_CI"]
        true_med_row = ["true median"]+[true_med]*num_repeat
        n_over_2_row = ["true n/2"] + [n/2]*num_repeat
        med_rank_row = ["true med_rank(find)"] + [med_rank]*num_repeat

        worksheet.append_row(data_uid_row)
        worksheet.append_row(est_median)
        worksheet.append_row(true_med_row)
        worksheet.append_row(original_errors)
        # write_to_sheet(worksheet1, errors, output_mode=output_mode, compare=compare)
        worksheet.append_row(ranks)
        worksheet.append_row(n_over_2_row)
        worksheet.append_row(rank_errors_1)
        # write_to_sheet(worksheet1, rank_errors_1, output_mode=output_mode, compare=compare)
        worksheet.append_row(med_rank_row)
        worksheet.append_row(rank_errors_2)
        worksheet.append_row(lengths)

        # write_to_sheet(worksheet1, rank_errors_2, output_mode=output_mode, compare=compare)
        worksheet.append_row(sorted_med)
        worksheet.append_row(sorted_ranks)
        worksheet.append_row(sorted_errors)
        worksheet.append_row(sorted_re_1)
        worksheet.append_row(sorted_re_2)
        worksheet.append_row(sorted_lengths)

      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
      if (output_mode == 3):
        output = row_data
        return output

In [24]:
def test_theirs(eps=eps, beta=beta, num_repeat=num_repeat, D=newD,median_method = 0,compare = False,output=[],sensitivity=0.5):
  #median_method = 0:median from CI algorithm
  #        = 1:median from EMMedianCI
  n = len(newD)
  domain_size = int((max(D)-min(D))/n)
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  ranks = []
  EM_median_errors = []
  med_rank = int(find_rank(rank_dict, true_med))
  est_list = []
  rank_errors_1 = [] # based on n/2
  rank_errors_2 = [] # based on find_rank(rank_dict,true_med)
  rank_CI_lengths = []
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      errors = []
      lengths = []
      correct_count = 0
      rad = 10000000
      left=-rad
      u,l,weight = constructu(eps[i], left*n,rad*n, newD,sensitivity=sensitivity)
      u1,weight1,u2,weight2,factor = constructu_CI(eps[i],beta[i], rad,u,l,sensitivity=sensitivity)
      for j in range(num_repeat):
          start = time.time()
          xl,xr,indi,est = EMMedianCI(l,weight,weight1,weight2,factor)
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          lengths.append((xr-xl)/(2))
          if (median_method == 0):
            their_median = (xr+xl)/2
          else:
            their_median = est
          est_median.append(their_median)
          errors.append(abs(their_median-true_med))
          EM_median_errors.append(abs(est-true_med)) # their def of errors
          rank = int(find_rank(rank_dict, their_median))
          ranks.append(rank)
          rank_errors_1.append(int(abs(n/2 - rank)))
          rank_errors_2.append(int(abs(med_rank - rank)))

          # rank CI length
          rank_lower = int(find_rank(rank_dict,xl))
          rank_upper = int(find_rank(rank_dict,xr))
          rank_CI_length = (rank_upper-rank_lower)/2
          rank_CI_lengths.append(rank_CI_length)

      original_errors = errors
      errors = np.array(errors, dtype=object)
      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]

      avgTime = sum(times)/len(times)


      errors = np.array(errors)
      avg_error = np.average(errors)
      if (errorQuantile == 0): relative_CI_width = -1
      else: relative_CI_width = avgLength / errorQuantile
      if np.all(errors == errors[0]):
          stddev_based_avg_err = errors[0]
          IQR_based_avg_err = errors[0]
      else:
          std_dev = np.std(errors)
          stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])

          q1 = np.percentile(errors, 25)
          q3 = np.percentile(errors, 75)
          iqr = q3 - q1
          lower_bound = q1 - 1.5 * iqr
          upper_bound = q3 + 1.5 * iqr
          IQR_based_avg_err = np.average(errors[(errors >= lower_bound) & (errors <= upper_bound)])

      #rank_error avg
      avg_rank_error_1 = np.average(rank_errors_1)
      avg_rank_error_2 = np.average(rank_errors_2)
      print("Filtered Errors for IQR_based_avg_err:", IQR_based_avg_err)
      # avg rank CI length
      avg_rank_CI_length = np.average(rank_CI_lengths)

      print("estimated median value: "+ str(np.average(est_median)))
      print("correct rate = " +str(correct_count/num_repeat))
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(avgLength/errorQuantile))
      print("Average Time to find CI = " + str(sum(times)/len(times)))
      # print("Errors:", errors)
      # print("Standard Deviation:", std_dev)
      # print("Filtered Errors for stddev_based_avg_err:", errors[np.abs(errors - avg_error) < 3 * std_dev])

      # print("Q1:", q1, "Q3:", q3)
      # print("IQR:", iqr)
      # print("Lower Bound:", lower_bound, "Upper Bound:", upper_bound)
      if (median_method == 0):
        name = "EM_CI"
      else:
        name = "EM"
      row_data = [
          name,
          data_uuid,
          n,
          domain_size, #3
          sens, #4
          beta[i],#5
          eps[i], #6
          num_repeat, #7
          true_med, #8
          np.average(est_median), #9
          correct_rate, #10
          avgLength, #11
          errorQuantile, #12
          avg_error,#13
          stddev_based_avg_err, #14
          IQR_based_avg_err, #15
          avg_rank_error_1, #16
          avg_rank_error_2, #17
          relative_CI_width, #18
          avg_rank_CI_length, #19
          avgTime
      ]
      if (output_mode == 0 or output_mode == 2):
        write_to_sheet(worksheet1, row_data, compare = compare)

      if (output_mode == 1 or output_mode == 2):
        sorted_med = sorted(est_median)
        est_median.insert(0, "EM est_median")
        sorted_med.insert(0, "Sorted EM est_median")

        sorted_errors = sorted(original_errors)
        original_errors.insert(0, "EM errors")
        sorted_errors.insert(0, "Sorted EM errors")

        sorted_ranks = sorted(ranks)
        ranks.insert(0, "EM ranks")
        sorted_ranks.insert(0, "Sorted EM ranks")

        sorted_re_1 = sorted(rank_errors_1)
        sorted_re_2 = sorted(rank_errors_2)
        rank_errors_1.insert(0, "EM rank_errors(n/2)")
        rank_errors_2.insert(0, "EM rank_errors(find)")
        sorted_re_1.insert(0, "Sorted EM rank_errors(n/2)")
        sorted_re_2.insert(0, "Sorted EM rank_errors(find)")

        sorted_lengths = sorted(lengths)
        lengths.insert(0,"EM CI lengths")
        sorted_lengths.insert (0, "Sorted EM CI lengths")

        data_uid_row = ["data:", data_uuid, "method:", "EM_CI"]
        true_med_row = ["true median"]+[true_med]*num_repeat
        n_over_2_row = ["true n/2"] + [n/2]*num_repeat
        med_rank_row = ["true med_rank(find)"] + [med_rank]*num_repeat

        print(est_list)
        worksheet.append_row(data_uid_row)
        worksheet.append_row(est_median)
        worksheet.append_row(true_med_row)

        worksheet.append_row(original_errors)
        # write_to_sheet(worksheet1, original_errors, output_mode=output_mode, compare=compare)
        worksheet.append_row(ranks)
        worksheet.append_row(n_over_2_row)
        worksheet.append_row(rank_errors_1)
        # write_to_sheet(worksheet1, rank_errors_1, output_mode=output_mode, compare=compare)
        worksheet.append_row(med_rank_row)
        worksheet.append_row(rank_errors_2)
        worksheet.append_row(lengths)
        # write_to_sheet(worksheet1, rank_errors_2, output_mode=output_mode, compare=compare)
        worksheet.append_row(sorted_med)
        worksheet.append_row(sorted_ranks)
        worksheet.append_row(sorted_errors)
        worksheet.append_row(sorted_re_1)
        worksheet.append_row(sorted_re_2)
        worksheet.append_row(sorted_lengths)



      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
      print(len_eps)
      print(err_eps)
      if (output_mode == 3):
        output = row_data
      return output

In [25]:
def readInput(mode,read_data_uuid="64b7c8bf89f14113bd12f5f15e276546"):
    import numpy as np
    file_paths = {
        0: ("./bank_marketing.csv", 5),
        1: ("./adult.csv", 2),
        2: ("./sample_data/california_housing_test.csv", 3),
        3: ("./sample_data/california_housing_test.csv", 8),
        5: ("./ratings_small.csv",3),
        6: ("./airplane_price_dataset.csv",1),
        7: ("./airplane_price_dataset.csv",2),
        8: ("./airplane_price_dataset.csv",3),
        9: ("./airplane_price_dataset.csv",4),
        10: ("./data_"+read_data_uuid+".csv",0),
        11: ("./healthcare.csv",1),
        12: ("./healthcare.csv",2),
        13: ("./Estate_Sales.csv",1),
        14: ("./Estate_Sales.csv",2)
    }

    if mode not in file_paths:
        raise ValueError("Invalid mode. Choose between 0 to 3")

    input_path, column_index = file_paths[mode]

    with open(input_path, 'r',errors='ignore') as input_file:
        lines = input_file.readlines()
    n = len(lines)
    # D = np.zeros(n)
    D = np.zeros(n-1) #new! there are n-1 data points
    for i, line in enumerate(lines):
        if i > 0:
            elements = line.split(",")
            try:
                value = float(elements[column_index])
                D[i - 1] = int(value)
            except ValueError:
              if (column_index < len(elements)):
                print(f"Skipping invalid data at line {i}: {elements[column_index]}")
    return D

def normalize_non_negative(D):
  if lowest<0:
    D = [d-lowest for d in D]
  return D

In [26]:
def color_vertically(worksheet,col=10,smaller_better=True,row_number=4):
  # row_number: number of rows you want to colour
  # Define the column indices
  col_true = 9
  values_I = worksheet.col_values(col_true)
  values_col = worksheet.col_values(col)

  # Extract the last k rows (ensuring they are numeric)
  last_4_true = [float(values_I[-row_number + i]) for i in range(row_number)]
  last_4_vals = [float(values_col[-row_number + i]) for i in range(row_number)]

  if (col==10):
    diffs = [abs(last_4_vals[i] - last_4_true[i]) for i in range(row_number)]
  else:
    diffs = last_4_vals

  # Rank differences from smallest to largest
  if smaller_better:
      sorted_indices = sorted(range(len(diffs)), key=lambda k: diffs[k])
  else:
      sorted_indices = sorted(range(len(diffs)), key=lambda k: diffs[k], reverse=True)


  # Define colors for ranking
  colors = [
    {"red": 0.0, "green": 1.0, "blue": 0.0},  # Green (best)
    {"red": 0.6, "green": 1.0, "blue": 0.6},  # Light Green
    {"red": 1.0, "green": 0.6, "blue": 0.6},  # Light Red
    {"red": 1.0, "green": 0.0, "blue": 0.0},  # Red (worst)
  ]

  # Get the last 4 row indices
  base_row = len(values_col) - row_number+1  # First of the last 4 rows (1-based index)

  # Apply formatting
  for rank, index in enumerate(sorted_indices):
    row = base_row + index  # Correct row number
    cell = f"{chr(64 + col)}{row}"  # Convert column number to letter (J)
    worksheet.format(cell, {"backgroundColor": colors[rank]})

In [27]:
def color_horizontally(worksheet,col_n=4,starting_col = 2):
    #starting_col starts from 1
    col_end = starting_col + col_n - 1

    # Get the last row values from Column B to Column E
    last_row = worksheet.row_values(len(worksheet.get_all_values()))
    # Convert the values to floats (or int if appropriate)
    values_b_to_end = [float(value) for value in last_row[starting_col - 1:col_end]]  # Ensure values are numeric

    # Define colors for ranking
    colors = [
        {"red": 0.0, "green": 1.0, "blue": 0.0},  # Green (min)
        {"red": 0.6, "green": 1.0, "blue": 0.6},  # Light Green (2nd smallest)
        {"red": 1.0, "green": 0.6, "blue": 0.6},  # Light Red
        {"red": 1.0, "green": 0.0, "blue": 0.0},  # Red (max)
    ]
    if col_n==3:
      colors = [
        {"red": 0.0, "green": 1.0, "blue": 0.0},  # Green (min)
        {"red": 0.6, "green": 1.0, "blue": 0.6},  # Light Green (2nd smallest
        {"red": 1.0, "green": 0.0, "blue": 0.0},  # Red (max)
      ]
    if col_n==5:
      colors = [
        {"red": 0.0, "green": 1.0, "blue": 0.0},  # Green (min)
        {"red": 0.6, "green": 1.0, "blue": 0.6},  # Light Green (2nd smallest)
        {"red": 1.0, "green": 1.0, "blue": 1.0},
        {"red": 1.0, "green": 0.6, "blue": 0.6},  # Light Red
        {"red": 1.0, "green": 0.0, "blue": 0.0},  # Red (max)
      ]

    if col_n == 7:
      colors = [
          {"red": 0.0, "green": 1.0, "blue": 0.0},  # Pure Green
          {"red": 0.4, "green": 0.8, "blue": 0.4},  # Darker Green
          {"red": 0.6, "green": 1.0, "blue": 0.6},  # Light Green
          {"red": 1.0, "green": 1.0, "blue": 1.0},  # White
          {"red": 1.0, "green": 0.8, "blue": 0.8},  # Light Red
          {"red": 1.0, "green": 0.5, "blue": 0.5},  # Darker Red
          {"red": 1.0, "green": 0.0, "blue": 0.0}   # Pure Red
      ]

    # Sort the values and map them to their ranks
    sorted_indices = sorted(range(len(values_b_to_end)), key=lambda k: values_b_to_end[k])
    rank_map = {sorted_indices[i]: i for i in range(len(sorted_indices))}  # Map index to rank

    # Get the column letters for Column B to Column E
    if (starting_col == 2):
      if col_n==4: columns = ['B', 'C', 'D', 'E']
      elif col_n==3: columns = ['B', 'C', 'D']
      elif col_n==5: columns = ['B', 'C', 'D', 'E', 'F']
      elif col_n==7: columns = ['B', 'C', 'D', 'E', 'F', 'G', 'H']
    elif (starting_col == 10):
      if col_n==4: columns = ['J', 'K', 'L', 'M']
      elif col_n==5: columns = ['J', 'K', 'L', 'M', 'N']
      elif col_n==7: columns = ['J', 'K', 'L', 'M', 'N','O','P']
    elif (starting_col == 6):
      if col_n==3: columns = ['F','G','H']
    elif (starting_col == 7):
      if col_n==4: columns = ['G','H','I','J']
    elif (starting_col == 8):
      if col_n==5: columns = ['H','I','J','K','L']
    elif (starting_col == 9):
      if col_n==4: columns = ['I','J','K','L']
    # Loop through each value and apply the background color based on its rank
    for i, value in enumerate(values_b_to_end):
        col_letter = columns[i]  # Get column letter
        cell = f"{col_letter}{len(worksheet.get_all_values())}"  # Cell reference
        rank = rank_map[i]  # Get the rank from the rank map
        worksheet.format(cell, {"backgroundColor": colors[rank]})


In [28]:
def build_rank_dict(D, discretized=False):
    n = len(D)
    vals, counts = np.unique(D, return_counts=True)
    rank_dict = {}
    rank = 0
    last_val = vals[-1] + (n if discretized else 1)
    vals = vals.tolist() + [int(last_val)]
    counts = counts.tolist() + [0]

    for i in range(len(vals) - 1):
        rank += counts[i]
        rank_dict[vals[i]] = rank
    rank_dict[vals[-1]] = rank

    return rank_dict


In [40]:
# Experiment Data Input

dataset_id_dict = {0:"bank_marketing", 1:"adult", 2:"cali_housing_test", 3:"cali_housing_value",
           5:"IMDB_time",6:"airplane_capacity",7:"airplane_range",8:"airplane_maintenance",9:"airplane_price",
           10: "old_synthetic_data",11:"healthcare_age",12:"healthcare_bill",13:"estate_assesval",14:"estate_saleamount"}
mode = 0
if (mode == -1):
  n = 4000  # 45#000
  domain_size = 100000
  D = np.random.uniform(0, domain_size+1, n)

  # normal distribution
  # n = 1000
  # mean = 1
  # std_dev = 1
  # D = np.random.normal(loc=mean, scale=std_dev, size=n) * 8000

  # gamma distribution
  # n = 10000
  # alpha = 0.5
  # beta_g = 5000
  # D = np.random.gamma(alpha, beta_g, n)

  # Laplace distribution
  # n = 10000
  # mean = 0
  # std_dev = 1000
  # D = np.random.laplace(loc=mean, scale=std_dev, size=n)

  # Exponential Distribution
  # scale = 2000
  # D = np.random.exponential(scale=scale, size=4000)

  # Beta Distribution
  # a = 1
  # b = 1
  # n = 2000
  # scale = 1000000
  # D = np.random.beta(a, b, n)
  # D = D * scale


  # Log Distribution
  # mean = 0
  # std_dev = 0.5
  # n = 3000
  # scale = 80000
  # D = np.random.lognormal(mean, std_dev, n) * scale

  # Combined distribution
  # D2 = np.random.beta(b, a, n)
  D = D.astype(int)


else:
  D = readInput(mode,read_data_uuid="2c368240d51540a9b0c2ab422f38ffdb")

lowest = min(D)
highest = max(D)
print("lowest,hightest:",lowest, highest)
if (lowest < 0):
  D = normalize_non_negative(D)

lowest = min(D)
highest = max(D)

df = pd.DataFrame(D)
domain_size = highest - lowest
n = len(D)

D.sort()
print(D)

D = discretize(D, 1)
newD = discrete(D)
# print(newD)
true_med = np.median(D)

print("Data_size:", n)
print("Domain size:", domain_size)
print("true median:", true_med)
# rank_dict


rank_dict= build_rank_dict(D)
rank_dict_discretized = build_rank_dict(newD,discretized=True)
rank_dict_record = LRUCache(capacity=100000)
rank_dict_record_discretized = LRUCache(capacity=100000)

# print("rank_dict_discretized:",rank_dict_discretized)
# keys = rank_dict.keys()
# ranks = rank_dict.values()
# print(next(iter(keys)),lowest,next(reversed(keys)),highest)

print("true median rank:",n/2)
use_discretized = 0
print("true median rank in rank_dict:",find_rank(rank_dict, true_med))
print("rank_dict:",rank_dict)
#print(find_rank(rank_dict, 51))

# Data Persistence
if (mode == -1):
  df = pd.DataFrame(D)
  data_uuid = uuid.uuid4().hex
  file_name = f"data_{data_uuid}.csv"
  df.to_csv(file_name, index=False, header=False)
  files.download(file_name)
  print(f"File saved as {file_name}")
else:
  data_uuid = dataset_id_dict[mode]


lowest,hightest: -8019.0 102127.0
[np.float64(0.0), np.float64(1172.0), np.float64(3962.0), np.float64(4647.0), np.float64(4706.0), np.float64(4961.0), np.float64(5192.0), np.float64(5307.0), np.float64(5415.0), np.float64(5737.0), np.float64(5897.0), np.float64(5926.0), np.float64(5937.0), np.float64(5970.0), np.float64(6039.0), np.float64(6051.0), np.float64(6054.0), np.float64(6075.0), np.float64(6078.0), np.float64(6135.0), np.float64(6135.0), np.float64(6137.0), np.float64(6165.0), np.float64(6201.0), np.float64(6238.0), np.float64(6240.0), np.float64(6273.0), np.float64(6282.0), np.float64(6289.0), np.float64(6294.0), np.float64(6318.0), np.float64(6339.0), np.float64(6351.0), np.float64(6355.0), np.float64(6358.0), np.float64(6364.0), np.float64(6383.0), np.float64(6390.0), np.float64(6398.0), np.float64(6398.0), np.float64(6406.0), np.float64(6418.0), np.float64(6421.0), np.float64(6433.0), np.float64(6472.0), np.float64(6474.0), np.float64(6488.0), np.float64(6519.0), np.float

In [42]:
# CI Parameters
eps = [0.25]
beta = [0.01]
sens = 1 #fixed
num_repeat = 1

# for val, rank in rank_dict.items():
#     print(val,"  ",rank)

# Parameters for our_CI
beta_1 = 0.5 *beta[0]
beta_2 = beta[0] - beta_1
# eps_1 = 0.5 * eps[0]
# eps_2 = eps[0] - eps_1
eps_1 = 0.5 * eps[0]
eps_2 = eps[0] - eps_1

print("γ1:",(2/eps_1) *(np.log(domain_size/beta_1))) # γ)

b_list_step = 1

download = 0

use_discretized = 1
show_EM_median_u = False
show_EMCI_u = False

γ1: 270.5260704640326


In [ ]:
# Run Before

In [ ]:
# Test both

# output_mode 0:experiment results,
#        1:medians&ranks, CI legnths
#        2:both
#        3:return row_result and do not write to google spreadsheet

# EM_mode  0: use our original EM
#       1: use our original EM with discretized data
#       2: use their EM with discretized data

output_mode = 3
download = 0
df = pd.DataFrame()
print("-------------------EMCI Test--------------------")
if show_EM_median_u:
  df["rank"] = [0]+list(range(1,len(D)+1)) + ['.']
  df["data"] = ['.']+list(D)+ ['.']
  df["discretized_data"] = ['.'] + list(newD) + [',']
eps_val = eps[0]
test_theirs(eps=[0.5*eps_val], beta=beta, num_repeat=num_repeat,median_method=1,D=newD,sensitivity=0.5) #test EM


print("-------------------ourCI Test--------------------")
optimized_step = int(2/eps_2)
use_discretized = 1
test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=1,sl_in_u=False)
# if (output_mode != 3):`
#   color_vertically(worksheet,col=10,row_number=4) #est_median
#   color_vertically(worksheet,col=14,row_number=4) #avg error
#   color_vertically(worksheet,col=12,row_number=4) #avgCI_length

# Result Persistence
csv_name = "weight"+data_uuid
df.to_csv(csv_name+".csv",index=False)
if (download == 1):
  files.download(csv_name+".csv")

-------------------EMCI Test--------------------
Results for EM based median, beta = 0.01
factor is 2101
Filtered Errors for IQR_based_avg_err: 1.0
estimated median value: 8468.0
correct rate = 1.0
Average CI length = 97.5
error quantile is 1.0
CI length/ error quantile = 97.5
Average Time to find CI = 0.011659622192382812
[97.5]
[np.float64(1.0)]
-------------------ourCI Test--------------------
Results for EM based median, beta = 0.01


In [ ]:
# Test any return index
# (use our EM_CI(discretized) and SVT_CI)
output_mode = 3
return_index = 10
experiment_target = "Correct Rate"
column1_title = "eps"
num_repeat = 10
use_opt_stepsize = False
b_list_step = 1
download = 0
show_EM_median_u = False
df = pd.DataFrame()
# worksheet.append_row(["alpha:"+str(alpha),"beta:"+str(beta_g)])
# worksheet.append_row(["mean:"+str(mean),"std_dev:"+str(std_dev)])
# worksheet.append_row(["scale:"+str(scale)])
eps_list = [0.25,0.5,1,2,4] #
if use_opt_stepsize:
  step_option = "opt step"
else:
  step_option = b_list_step
worksheet.append_row([data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),experiment_target,"b_step:"+str(step_option),"num_repeat:"+str(num_repeat)])
worksheet.append_row([column1_title,"their_EM","our_EM","our_EM(sl_in_u)"])


for eps_val in eps_list:
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=1,D=newD,sensitivity=0.5)[return_index]
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  if use_opt_stepsize:
    b_list_step = optimized_step

  use_discretized = 1
  ourCI_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=1,sl_in_u=False)[return_index]

  # ourCI2_output = 0
  ourCI2_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=1,sl_in_u=True)[return_index]
  output_RowData = [eps_val,EMCI_output,ourCI_output,ourCI2_output]
  worksheet.append_row(output_RowData)
  # color_horizontally(worksheet)



In [ ]:
# Test both Median err and CI
# including eps split
output_mode = 3
num_repeat=100
CIindex = 11
errindex = 13
b_list_step = 1
CI_method = 1

first_row_err = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg Median err","b_step:"+str(b_list_step),"num_repeat:"+str(num_repeat)]
first_row_CI = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg CI length","b_step:"+str(b_list_step),"num_repeat:"+str(num_repeat)]
test_types = ["eps","EM_CI","our_CI(b_step)","our_CI(opt)","our_CI 4:6","our_CI(opt) 4:6","our_CI 6:4","our_CI(opt) 6:4"]
worksheet.append_row(first_row_err+first_row_CI)
worksheet.append_row(test_types+test_types)
eps_list = [0.25,0.5,1,2,4]
df = pd.DataFrame()

for eps_val in eps_list:
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=0,D=newD)
  EMCI_err = EMCI_output[errindex]
  EMCI_CI = EMCI_output[CIindex]

  # even split eps
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(8/eps_2)
  use_discretized = 0
  ourCI_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D,CI_method=CI_method)
  ourCI2_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D,CI_method=CI_method)
  ourCI_err = ourCI_output[errindex]
  ourCI_CI = ourCI_output[CIindex]
  ourCI2_err = ourCI2_output[errindex]
  ourCI2_CI = ourCI2_output[CIindex]

  # eps1:eps2 = 4:6
  eps_1 = 0.4 * eps_val
  eps_2 = eps_val - eps_1
  ourCI3_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  ourCI4_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  ourCI3_err = ourCI3_output[errindex]
  ourCI3_CI = ourCI3_output[CIindex]
  ourCI4_err = ourCI4_output[errindex]
  ourCI4_CI = ourCI4_output[CIindex]

  # eps1:eps2 = 6:4
  eps_1 = 0.6 * eps_val
  eps_2 = eps_val - eps_1
  ourCI5_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  ourCI6_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  ourCI5_err = ourCI5_output[errindex]
  ourCI5_CI = ourCI5_output[CIindex]
  ourCI6_err = ourCI6_output[errindex]
  ourCI6_CI = ourCI6_output[CIindex]


  # use_discretized = 1
  # ourCI2_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD)[return_index] #our_CI(discretized)
  # ourCI3_output = test_ours(b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,med_method=1)[return_index] #our_CI(their_EM)
  RowData_err = [eps_val,EMCI_err,ourCI_err,ourCI2_err,ourCI3_err,ourCI4_err,ourCI5_err,ourCI6_err]
  RowData_CI = [eps_val,EMCI_CI,ourCI_CI,ourCI2_CI,ourCI3_CI,ourCI4_CI,ourCI5_CI,ourCI6_CI]
  Final_RowData = RowData_err+RowData_CI
  worksheet.append_row(Final_RowData)
  color_horizontally(worksheet,col_n=7)
  color_horizontally(worksheet,col_n=7,starting_col=10)





In [ ]:
# Test both Median err and CI
# no eps split
output_mode = 3
num_repeat = 1
CIindex = 11
# errindex = 13 # Est Median error
errindex = 10 # this is actually correct rate
b_list_step = 1

# worksheet.append_row(["a="+str(a),"b="+str(b),"scale="+str(scale),"true_med:"+str(true_med)]) # beta distribution parameters
first_row_err = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"correct rate","b_step:opt","num_repeat:"+str(num_repeat)]
first_row_CI = ["avg CI length"]
test_types = ["eps","EM_CI","our_CI","our_CI(sl_in_u)"]
worksheet.append_row(first_row_err+first_row_CI)
worksheet.append_row(test_types+test_types)
eps_list = [0.25,0.5,1,2]
df = pd.DataFrame()

for eps_val in eps_list:
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=0,D=newD)
  EMCI_err = EMCI_output[errindex]
  EMCI_CI = EMCI_output[CIindex]

  # even split eps
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  use_discretized = 1
  ourCI_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=1,sl_in_u=False)
  ourCI2_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=1,sl_in_u=True)
  ourCI_err = ourCI_output[errindex]
  ourCI_CI = ourCI_output[CIindex]
  ourCI2_err = ourCI2_output[errindex]
  ourCI2_CI = ourCI2_output[CIindex]

  RowData_err = [eps_val,EMCI_err,ourCI_err,ourCI2_err]
  RowData_CI = [eps_val,EMCI_CI,ourCI_CI,ourCI2_CI]
  Final_RowData = RowData_err+RowData_CI
  worksheet.append_row(Final_RowData)
  # color_horizontally(worksheet,col_n=3)
  color_horizontally(worksheet,col_n=3,starting_col=6)





---------Runing eps:0.25---------
Results for EM based median, beta = 0.01
factor is 1050
Filtered Errors for IQR_based_avg_err: 2.0
estimated median value: 8469.0
correct rate = 1.0
Average CI length = 51.0
error quantile is 2.0
CI length/ error quantile = 25.5
Average Time to find CI = 0.06326079368591309
[51.0]
[np.float64(2.0)]
Results for EM based median, beta = 0.01


In [ ]:
# Test CI with different b_step
output_mode = 3
return_index = 11
experiment_target = "Avg CI Length"
column1_title = "eps"
b_list_step = 50
num_repeat = 20
CI_method = 1
# worksheet.append_row(["alpha:"+str(alpha),"beta:"+str(beta_g)])
# worksheet.append_row(["mean:"+str(mean),"std_dev:"+str(std_dev),"scale="+str(scale)])
# worksheet.append_row(["a="+str(a),"b="+str(b),"scale="+str(scale),"true_med:"+str(true_med)]) # beta distribution parameters
worksheet.append_row([data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"Avg CI Length","b_step:"+str(b_list_step),"num_repeat:"+str(num_repeat),"true_med:"+str(true_med)])
worksheet.append_row([column1_title,"EM_CI","our_CI(b_step)","our_CI(opt)","our_CI(opt/2)","our_CI(opt*2)"])
# eps_list = [0.25,0.5,1,2] #
eps_list = [1,2]

df = pd.DataFrame()

for eps_val in eps_list:
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=0,D=newD)[return_index] #test EM_CI
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  use_discretized = 1
  ourCI_output = test_ours(D=newD,b_list_step=b_list_step,eps=eps, beta=beta, num_repeat=num_repeat,compare = True,CI_method=CI_method )[return_index]
  ourCI_output2 = test_ours(D=newD,b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,compare = True,CI_method=CI_method )[return_index]
  ourCI_output3 = test_ours(D=newD,b_list_step=int(optimized_step/2),eps=eps, beta=beta, num_repeat=num_repeat,compare = True,CI_method=CI_method )[return_index]
  ourCI_output4 = test_ours(D=newD,b_list_step=optimized_step*2,eps=eps, beta=beta, num_repeat=num_repeat,compare = True,CI_method=CI_method )[return_index]
  output_RowData = [eps_val,EMCI_output,ourCI_output,ourCI_output2,ourCI_output3,ourCI_output4]
  worksheet.append_row(output_RowData)
  color_horizontally(worksheet,col_n=5)


In [ ]:
# def find_best_eps_splitting(domain_size,eps_val):
#   r_values = np.arange(0.01, 1, 0.01)
#   min_error_bound = float('inf')
#   min_r_value = None
#   min_error = None
#   for r in r_values:
#     eps_1 = float(r * eps_val)
#     eps_2 = float(eps_val - eps_1)
#     optimized_step = int(8/eps_2)
#     b_list_step = optimized_step
#     b_list = range(0, b_list_step*int(domain_size/b_list_step),b_list_step)
#     if (len(b_list) == 0): b_list = range(0,1)
#     C = (2/eps_1) *(np.log(domain_size/beta_1))
#     m = len(b_list)
#     alpha = (8*np.log(m)+np.log(2/beta_2)) / eps_2
#     error_bound = C + alpha + optimized_step
#     if error_bound < min_error_bound:
#       min_error_bound = error_bound
#       min_r_value = r
#       min_error = error_bound

#   return min_r_value
def find_best_eps_splitting(domain_size,eps_val,b_list_step=0):
  r_values = np.arange(0.01, 1, 0.01)
  min_error_bound = float('inf')
  min_r_value = None
  for r in r_values:
    eps_1 = float(r * eps_val)
    eps_2 = float(eps_val - eps_1)
    optimized_step = int(2/eps_2)
    b_list_step = optimized_step
    if (b_list_step == 0): b_list_step = 1
    # b_list_step = 1
    b_list = range(0, b_list_step*int(domain_size/b_list_step),b_list_step)
    if (len(b_list) == 0): b_list = range(0,1)
    C1 = (2/eps_1) *(np.log(domain_size/beta_1))
    C2 = (1/eps_2) *(np.log(domain_size/(b_list_step*beta_2)))
    m = len(b_list)
    error_bound = C1 + C2 + 1*b_list_step
    if error_bound < min_error_bound:
      min_error_bound = error_bound
      min_r_value = r

  return min_r_value

In [ ]:
# Test both Median err and CI
# eps split (using opt_b)
output_mode = 3
num_repeat = 3
CI_method = 1
download = 0
eps_list = [0.25,0.5,1,2]
eps_list = [1]
use_rank = False
if use_rank:
  CIindex = 19
  errindex = 17
else:
  CIindex = 11
  errindex = 13

first_row_err = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg Median err","num_repeat:"+str(num_repeat)]
first_row_CI = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg CI length","num_repeat:"+str(num_repeat)]
# test_types = ["eps","EM_CI","our_CI 7:3","our_CI 6:4","our_CI 5:5","our_CI 4:6","our_CI 3:7","our_CI(best)"]
test_types = ["eps","EM_CI","our_CI 6:4","our_CI 5:5","our_CI 4:6","our_CI best"]
worksheet.append_row(first_row_err+first_row_CI)
worksheet.append_row(test_types+test_types)

df = pd.DataFrame()
for eps_val in eps_list:
  # time.sleep(10)
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=0,D=newD)
  EMCI_err = EMCI_output[errindex]
  EMCI_CI = EMCI_output[CIindex]

  use_discretized = 1
  # 7:3
  # eps_1 = 0.7 * eps_val
  # eps_2 = eps_val - eps_1
  # optimized_step = int(8/eps_2)
  # ourCI_output = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  # ourCI_err = ourCI_output[errindex]
  # ourCI_CI = ourCI_output[CIindex]

  # 6:4
  eps_1 = 0.6 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  ourCI_output2 = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=CI_method)
  ourCI_err2 = ourCI_output2[errindex]
  ourCI_CI2 = ourCI_output2[CIindex]

  # 5:5
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  ourCI_output3 = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=CI_method)
  ourCI_err3 = ourCI_output3[errindex]
  ourCI_CI3 = ourCI_output3[CIindex]

  # 4:6
  eps_1 = 0.4 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  ourCI_output4 = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=CI_method)
  ourCI_err4 = ourCI_output4[errindex]
  ourCI_CI4 = ourCI_output4[CIindex]

  # 3:7
  # eps_1 = 0.3 * eps_val
  # eps_2 = eps_val - eps_1
  # optimized_step = int(8/eps_2)
  # ourCI_output5 = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  # ourCI_err5 = ourCI_output5[errindex]
  # ourCI_CI5 = ourCI_output5[CIindex]

  # best eps splitting
  r = find_best_eps_splitting(domain_size=domain_size,eps_val=eps_val,b_list_step=optimized_step)
  eps_1 = r * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(2/eps_2)
  ourCI_output6 = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD,CI_method=CI_method)
  ourCI_err6 = ourCI_output6[errindex]
  ourCI_CI6 = ourCI_output6[CIindex]
  # RowData_err = [eps_val,EMCI_err,ourCI_err,ourCI_err2,ourCI_err3,ourCI_err4,ourCI_err5,ourCI_err6]
  # RowData_CI = [eps_val,EMCI_CI,ourCI_CI,ourCI_CI2,ourCI_CI3,ourCI_CI4,ourCI_CI5,ourCI_CI6]
  RowData_err = [eps_val,EMCI_err,ourCI_err2,ourCI_err3,ourCI_err4,ourCI_err6]
  RowData_CI = [eps_val,EMCI_CI,ourCI_CI2,ourCI_CI3,ourCI_CI4,ourCI_CI6]
  Final_RowData = RowData_err+RowData_CI
  worksheet.append_row(Final_RowData)
  color_horizontally(worksheet,col_n=5)
  color_horizontally(worksheet,col_n=5,starting_col=8)




In [ ]:
# Test both Median err and CI
# discretized
output_mode = 0
num_repeat=100
eps_list = [0.25,0.5,1,2,4]
use_rank = False
if use_rank:
  CIindex = 19
  errindex = 17
else:
  CIindex = 11
  errindex = 13

worksheet.append_row(["value:"])
first_row_err = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg Median err","num_repeat:"+str(num_repeat)]
first_row_CI = ["avg CI length","num_repeat:"+str(num_repeat)]
test_types = ["eps","EM_CI","our_CI","our_CI(discre)"]
worksheet.append_row(first_row_err+first_row_CI)
worksheet.append_row(test_types+test_types)

df = pd.DataFrame()


EMCI_output = {key: [] for key in eps_list}
ourCI_output = {key: [] for key in eps_list}
ourCI_output2 = {key: [] for key in eps_list}


for eps_val in eps_list:
  time.sleep(10)
  eps = [eps_val]
  print("---------Runing eps:"+str(eps_val)+"---------")
  EMCI_output[eps_val] = test_theirs(eps=eps, beta=beta, num_repeat=num_repeat,median_method=0,D=newD)
  EMCI_err = EMCI_output[eps_val][errindex]
  EMCI_CI = EMCI_output[eps_val][CIindex]

  use_discretized = 0
  eps_1 = 0.5 * eps_val
  eps_2 = eps_val - eps_1
  optimized_step = int(8/eps_2)
  ourCI_output[eps_val]  = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=D)
  ourCI_err = ourCI_output[eps_val][errindex]
  ourCI_CI = ourCI_output[eps_val][CIindex]

  use_discretized = 1
  ourCI_output2[eps_val] = test_ours(b_list_step=optimized_step,eps=eps, beta=beta, num_repeat=num_repeat,D=newD)
  ourCI_err2 = ourCI_output2[eps_val][errindex]
  ourCI_CI2 = ourCI_output2[eps_val][CIindex]

  RowData_err = [eps_val,EMCI_err,ourCI_err,ourCI_err2]
  RowData_CI = [eps_val,EMCI_CI,ourCI_CI,ourCI_CI2]
  Final_RowData = RowData_err+RowData_CI
  worksheet.append_row(Final_RowData)
  color_horizontally(worksheet,col_n=3)
  color_horizontally(worksheet,col_n=3,starting_col=6)

use_rank = True
worksheet.append_row(["rank:"])
first_row_err = [data_uuid,"domain_size:"+str(domain_size),"n:"+str(n),"avg Median err","num_repeat:"+str(num_repeat)]
first_row_CI = ["avg CI length","num_repeat:"+str(num_repeat)]
test_types = ["eps","EM_CI","our_CI","our_CI(discre)"]
worksheet.append_row(first_row_err+first_row_CI)
worksheet.append_row(test_types+test_types)

if use_rank:
  CIindex = 19
  errindex = 17
else:
  CIindex = 11
  errindex = 13
for eps_val in eps_list:
  time.sleep(10)

  EMCI_err = EMCI_output[eps_val][errindex]
  EMCI_CI = EMCI_output[eps_val][CIindex]

  ourCI_err = ourCI_output[eps_val][errindex]
  ourCI_CI = ourCI_output[eps_val][CIindex]

  ourCI_err2 = ourCI_output2[eps_val][errindex]
  ourCI_CI2 = ourCI_output2[eps_val][CIindex]

  RowData_err = [eps_val,EMCI_err,ourCI_err,ourCI_err2]
  RowData_CI = [eps_val,EMCI_CI,ourCI_CI,ourCI_CI2]
  Final_RowData = RowData_err+RowData_CI
  worksheet.append_row(Final_RowData)
  color_horizontally(worksheet,col_n=3)
  color_horizontally(worksheet,col_n=3,starting_col=6)



In [ ]:
  # Test error bound with diff eps splitting for all datasets
import matplotlib.pyplot as plt
dataset_id_dict = {0:"bank_marketing", 1:"adult", 2:"cali_housing_test", 3:"cali_housing_value",
           5:"IMDB_time",6:"airplane_capacity",7:"airplane_range",8:"airplane_maintenance",9:"airplane_price",
           10: "old_synthetic_data",11:"healthcare_age",12:"healthcare_bill",13:"estate_assesval",14:"estate_saleamount"}
eps_list = [0.25,0.5,1,2,4]
for mode,data_uuid in dataset_id_dict.items():
  if mode == 0:
    D = discrete(readInput(mode))
    n = len(D)
    lowest = int(min(D))
    highest = int(max(D))
    domain_size = highest - lowest

    for eps_val in eps_list:
      error_bound_values = []
      eps = [eps_val]
      r_values = np.arange(0.01, 1, 0.01)
      min_error_bound = float('inf')
      min_r_value = None
      min_error = None
      for r in r_values:
        eps_1 = float(r * eps_val)
        eps_2 = float(eps_val - eps_1)
        optimized_step = int(2/eps_2)
        b_list_step = optimized_step
        l = 1
        if (b_list_step == 0): b_list_step = 1
        # b_list_step = 1
        b_list = range(0, b_list_step*int(domain_size/b_list_step),b_list_step)
        if (len(b_list) == 0): b_list = range(0,1)
        C1 = (2/eps_1) *(np.log(domain_size/beta_1))
        C2 = (1/eps_2) *(np.log(domain_size/(b_list_step*beta_2)))
        m = len(b_list)
        if (m == 0): print("m=0, r=:",r)
        error_bound = C1 + C2 + l * optimized_step
        error_bound_values.append(error_bound)
        if error_bound < min_error_bound:
          min_error_bound = error_bound
          min_r_value = r
          min_error = error_bound
      # Plot
      plt.plot(r_values, error_bound_values, label=data_uuid)
      plt.scatter(min_r_value, min_error, color='red', zorder=5)
      plt.annotate(f"Min: {min_error:.2f} at r:{min_r_value}",
              (min_r_value, min_error),
              textcoords="offset points",
              xytext=(0,10),
              ha='center', color='red')


      plt.title(f"Error Bound vs r for Dataset: {data_uuid} with eps:{eps_val}")
      plt.xlabel("r")
      plt.ylabel("Error Bound")
      plt.legend(loc="best")
      plt.grid(True)
      plt.show()

In [ ]:
# Test error bound with diff domain_size
import matplotlib.pyplot as plt
dataset_id_dict = {0:"bank_marketing", 1:"adult", 2:"cali_housing_test", 3:"cali_housing_value",
           5:"IMDB_time",6:"airplane_capacity",7:"airplane_range",8:"airplane_maintenance",9:"airplane_price",
           10: "old_synthetic_data",11:"healthcare_age",12:"healthcare_bill",13:"estate_assesval",14:"estate_saleamount"}
eps_list = [0.25,0.5,1,2,4]
domain_sizes = range(1000,100000,1)
error_bound_values = []
min_error_bound = float('inf')
min_domain = None
min_error = None
r = 0.6
eps_val = 2
l = 1
eps_1 = float(r * eps_val)
eps_2 = float(eps_val - eps_1)
for domain_size in domain_sizes:
  optimized_step = int(2/eps_2)
  b_list_step = optimized_step
  # b_list_step = 1
  b_list = range(b_list_step, b_list_step*int(domain_size/b_list_step),b_list_step)
  C1 = (2/eps_1) *(np.log(domain_size/beta_1))
  C2 = (1/eps_2) *(np.log(domain_size/(b_list_step*beta_2)))
  m = len(b_list)
  # alpha = (8*np.log(m)+np.log(2/beta_2)) / eps_2
  if (m == 0): print("m=0, r=:",r)
  error_bound = C1 + C2 + b_list_step
  error_bound_values.append(error_bound)
  if error_bound < min_error_bound:
    min_error_bound = error_bound
    min_domain = domain_size
    min_error = error_bound
plt.plot(domain_sizes, error_bound_values, label=data_uuid)
plt.scatter(min_domain, min_error, color='red', zorder=5)



plt.title(f"Error Bound vs domain_size")
plt.xlabel("domain_size")
plt.ylabel("Error Bound")
plt.grid(True)
plt.show()

In [ ]:
# see eps splitting
import math

N = 1000
beta1 = 0.005
beta2 = 0.005
epsilon = 1
max_iterations = 100
# s = 2/(2/3)
s = 1
def calculate_epsilon_1(epsilon_2, N, beta1, beta2, s):
  return epsilon_2 * math.sqrt(math.log(N / beta1) / math.log(N / (s * beta2)))


def iterative_optimization(N, beta1, beta2, epsilon, max_iterations):
  global s
  iteration = 0
  while iteration < max_iterations:
    denominator = 1 + math.sqrt(math.log(N / beta1) / math.log(N / (s * beta2)))
    epsilon_2 = epsilon / denominator
    epsilon_1 = calculate_epsilon_1(epsilon_2, N, beta1, beta2, s)
    epsilon_2 = epsilon_2
    s = int(2/epsilon_2)
    iteration += 1
    # print(s)
    # print(epsilon_1,epsilon_2)
  return epsilon_1, epsilon_2

D = discrete(readInput(0))
n = len(D)
lowest = int(min(D))
highest = int(max(D))
domain_size = highest - lowest
eps_1,eps_2 = iterative_optimization(N, beta1, beta2, epsilon, max_iterations)
print(eps_1,eps_2)
C1 = (2/eps_1) *(np.log(domain_size/beta_1))
C2 = (1/eps_2) *(np.log(domain_size/(b_list_step*beta_2)))
b_list_step = 2/eps_2
error_bound = C1 + C2 + b_list_step
print(error_bound)

In [ ]:
D = [25,50,75]
newD = discrete(D)
rank_dict, rank_dict_record = build_rank_dict(D)
rank_dict_discretized,rank_dict_record_discretized = build_rank_dict(newD,discretized=True)


use_discretized = 0
print("test 1:")
print("rank_dict:",rank_dict)
for i in range(101):
  res = find_rank(rank_dict,i)
  print(i,res)

use_discretized = 1
print("test 2:")
print("rank_dict_discretized:",rank_dict_discretized)
for i in range(401):
  res = find_rank(rank_dict_discretized,i)
  print(i,i//4,res)


test 1:
rank_dict: {25: 1, 50: 2, 75: 3, 76: 3}
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 2
51 2
52 2
53 2
54 2
55 2
56 2
57 2
58 2
59 2
60 2
61 2
62 2
63 2
64 2
65 2
66 2
67 2
68 2
69 2
70 2
71 2
72 2
73 2
74 2
75 3
76 3
77 3
78 3
79 3
80 3
81 3
82 3
83 3
84 3
85 3
86 3
87 3
88 3
89 3
90 3
91 3
92 3
93 3
94 3
95 3
96 3
97 3
98 3
99 3
100 3
test 2:
rank_dict_discretized: {75.0: 1, 150.0: 2, 225.0: 3, 228: 3}
0 0 0
1 0 0
2 0 0
3 0 0
4 1 0
5 1 0
6 1 0
7 1 0
8 2 0
9 2 0
10 2 0
11 2 0
12 3 0
13 3 0
14 3 0
15 3 0
16 4 0
17 4 0
18 4 0
19 4 0
20 5 0
21 5 0
22 5 0
23 5 0
24 6 0
25 6 0
26 6 0
27 6 0
28 7 0
29 7 0
30 7 0
31 7 0
32 8 0
33 8 0
34 8 0
35 8 0
36 9 0
37 9 0
38 9 0
39 9 0
40 10 0
41 10 0
42 10 0
43 10 0
44 11 0
45 11 0
46 11 0
47 11 0
48 12 0
49 12 0
50 12 0
51 12 0
52 13 0
53 13 0
54 13